# Tuning a Semantic Segmentation Transformers

We have now learned the power of the transformer architecture.
One of its applications is to perform semantic segmentation.

Althgouh training a full transformer from scratch can be expensive, it is relatively easy to "tune" a pre-trained transformer to different types of data.